In [5]:
from itertools import groupby
import pandas as pd
from pathlib import Path
OUT_DIR = Path(r"C:\Users\Admin\Downloads\curs-coders-lab\Projects\fifa_players_analysis\outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Base project folder
DATA_DIR = Path(r"C:\Users\Admin\Downloads\curs-coders-lab\Projects\fifa_players_analysis")

# Load only first 10k rows from male_players file from Kaggle data: https://www.kaggle.com/datasets/stefanoleone992/fifa-23-complete-player-dataset?select=male_players.csv
df = pd.read_csv(DATA_DIR / "male_players.csv", encoding="utf-8", nrows=10000)

# Save as a new file inside the project folder
output_path = DATA_DIR / "male_players_10k_rows.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print("Saved new file:", output_path)

# Inspect
print("Rows,Cols:", df.shape)
print(df.columns.tolist()[:50])

# Basic cleaning
df=df.copy()
df.columns = df.columns.str.strip()

# Select the useful columns
possible_cols = ["pace", "shooting", "passing", "dribbling", "defending", "physic", "attacking_crossing",
 "attacking_finishing", "attacking_heading_accuracy", "short_name", "long_name", 
 "player_positions", "overall", "potential", "value_eur", "wage_eur", "age", "height_cm", "weight_kg",
 "league_name","club_name","nationality_name","skill_moves","release_clause_eur","preferred_foot"]

existing = [c for c in possible_cols if c in df.columns]
df = df[existing].copy()
print("Using columns:", existing)

# Convert numeric columns
numeric_cols = [
    "pace", "shooting", "passing", "dribbling", "defending", "physic",
    "attacking_crossing", "attacking_finishing", "attacking_heading_accuracy",
    "overall", "potential", "value_eur", "wage_eur", "age",
    "height_cm", "weight_kg", "release_clause_eur", "skill_moves"
]

for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Impute numeric missing values (MEDIAN)
for c in numeric_cols:
    if c in df.columns:
        df[c].fillna(df[c].median(), inplace=True)

# Weighted composite score 
weights ={
"overall": 0.40,
"pace":0.12,
"shooting":0.12,
"passing":0.12,
"dribbling":0.12,
"defending":0.06,
"physic":0.06,}

def computre_score (row):
    s=0.0
    total_w= 0.0
    for k,w in weights.items():
        if k in row.index and pd.notnull(row[k]):
            s+=row[k]*w
            total_w+= w
    return s/total_w if total_w>0 else 0

df["composite_score"] = df.apply(computre_score, axis=1)


#Rank players
df=df.sort_values(by=["composite_score","overall"], ascending=False).reset_index(drop=True)
df["rank_overall"] = df["overall"].rank(method="dense",ascending=False).astype(int)
df["rank_composite"] = df["composite_score"].rank(method="dense",ascending=False).astype(int)


#Top 20 plauers 
name_col = "short_name" if "short_name" in  df.columns else "long_name"
team_col = "club_name" if "club_name" in df.columns else "team"

top20 = df.head(20)[[name_col,"age", team_col,"overall","composite_score","player_positions","value_eur",
"wage_eur", "league_name","nationality_name"]]
top20.to_csv(OUT_DIR / "top20_players.csv", index=False)
print(" Saved top 20 players.csv")

#Full cleaned dataset for SQL or Tableau
df.to_csv(OUT_DIR / "cleaned_players.csv", index=False)
print("Saved cleaned players.csv")

#aggregate team-level averages 
if team_col in df.columns:
    team_agg = (df.groupby(team_col).agg(avg_overall=("overall","mean")
    ,avg_composite=("composite_score","mean"),
    count_players=(name_col,"count"))
    .reset_index()
    .sort_values("avg_composite",ascending=False))
    team_agg.to_csv(OUT_DIR/"teams_summary.csv",index=False)

    print("Saved teams_summary.csv")
    print("\n All outputs saved in :", OUT_DIR)

Saved new file: C:\Users\Admin\Downloads\curs-coders-lab\Projects\fifa_players_analysis\male_players_10k_rows.csv
Rows,Cols: (10000, 110)
['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy']
Using columns: ['pace', 'shooting', 'passing', 'dribb

C:\Users\Admin\AppData\Local\Temp\ipykernel_22092\2482495228.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22092\2482495228.py:52: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

 Saved top 20 players.csv
Saved cleaned players.csv
Saved teams_summary.csv

 All outputs saved in : C:\Users\Admin\Downloads\curs-coders-lab\Projects\fifa_players_analysis\outputs
